In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import seaborn as sns
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
import random

### Data Loading

In [2]:
out_dir = Path("../phys-vae/data/climate/")
data = pd.read_parquet("complete_data.parquet")

### Cleaning and NaN removal

In [3]:
# Replace -9999 with NaN
data['NEE'] = data['NEE'].replace(-9999, np.nan)

# Identify the consecutive gaps
data['Gap'] = data['NEE'].isna().astype(int).groupby(data['NEE'].notna().astype(int).cumsum()).cumsum()

# Filter out the zero values and calculate the lengths of the gaps
gap_lengths = data[data['Gap'] != 0]['Gap'].value_counts().reset_index()
gap_lengths.columns = ['Gap Length', 'Frequency']

gap_lengths

,Gap Length,Frequency
0,1,8619
1,2,4881
2,3,3767
3,4,3013
4,5,2540
...,...,...
810,751,1
811,749,1
812,748,1
813,747,1


In [4]:
data = data.dropna().reset_index(drop=True)

### Adding features

In [5]:
def set_stats(df, flux, scale="30T"):
    df = df.copy()
    ind = df.index
    dt = df["DateTime"]
    df.set_index('DateTime', inplace=True)
    flux_orig = df[flux].copy()
    df[flux] = df[flux].interpolate().values
    
    # Set max
    flux_max = df[flux].resample("D").max()
    df["flux_max"] = flux_max.resample(scale).bfill()
    
    # Set min
    flux_min = df[flux].resample("D").min()
    df["flux_min"] = flux_min.resample(scale).bfill()
    
    # Set mean
    flux_mean = df[flux].resample("D").mean()
    df["flux_mean"] = flux_mean.resample(scale).bfill()
    
    # Set std
    flux_std = df[flux].resample("D").std()
    df["flux_std"] = flux_std.resample(scale).bfill()
    
    # Set 25%, 50%, 75% quantiles
    flux_p25 = df[flux].resample("D").quantile(0.25)
    df["flux_p25"] = flux_p25.resample(scale).bfill()
    
    flux_p50 = df[flux].resample("D").quantile(0.50)
    df["flux_p50"] = flux_p50.resample(scale).bfill()
    
    flux_p75 = df[flux].resample("D").quantile(0.75)
    df["flux_p75"] = flux_p75.resample(scale).bfill()
    
    df = df.interpolate()
    
    df.index = ind
    df["DateTime"] = dt.values
    df.loc[:, flux] = flux_orig.values
    return df, ["flux_max", "flux_min", "flux_mean", "flux_std", "flux_p25", "flux_p50", "flux_p75"]
 
#====================================================================================================
# set season tag
def set_season_tag(df, isnorth = True):
    if isnorth:
        df["season"] = (df['DateTime'].month%12 + 3) // 3 # print(seasons)
    else:
        df["season"] = ((df['DateTime'].month + 6)%12 + 3)//3
    return df, ["season"]
#====================================================================================================
# set radiance tag

def set_rg_tag(df, rg):
    df["rg_rank"] = np.select(
        condlist = [
            df[rg] < 10,
            (df[rg] > 10) & (df[rg] < 100),
            df[rg] > 100
        ],
        choicelist = [
            1,
            2,
            3
        ],
        default = 0
    )
    return df, ["rg_rank"]

def extract_features(data):
    """
    Extract features from the timestamp for model training.
    
    Parameters:
    data (pd.DataFrame): DataFrame with 'Timestamp' and 'NEE' columns.
    
    Returns:
    pd.DataFrame: DataFrame with extracted features.
    """
    data = data.copy()
    data['hour'] = data['DateTime'].dt.hour
    data['dayofweek'] = data['DateTime'].dt.dayofweek
    data['month'] = data['DateTime'].dt.month
    data['dayofyear'] = data['DateTime'].dt.dayofyear
    return data

In [6]:
data, stat_tags = set_stats(data, "NEE")
data, rg_tag = set_rg_tag(data, 'Rg')

In [7]:
data, stat_tags = set_stats(data, "NEE")
data, rg_tag = set_rg_tag(data, 'Rg')
data = extract_features(data)

In [8]:
data.head(10)

,NEE,NEE_unc,LE,LE_unc,H,H_unc,Tau,Tau_unc,CO2_strg,LE_strg,...,flux_std,flux_p25,flux_p50,flux_p75,rg_rank,DateTime,hour,dayofweek,month,dayofyear
0,12.5670,3.06210,-9999.000,14.6310,-2.9117,0.72031,0.17393,0.012095,-0.103920,-0.512280,...,0.986102,4.82125,5.3522,5.86985,1,2012-06-22 00:30:00,0,4,6,174
1,6.7892,2.06360,37.362,12.3280,-10.5920,0.64084,0.17662,0.012124,0.089560,-0.189890,...,0.986102,4.82125,5.3522,5.86985,1,2012-06-22 01:00:00,1,4,6,174
2,4.0302,0.54104,35.169,4.7136,-15.9840,1.20660,0.18260,0.013019,-0.204920,-0.404700,...,0.986102,4.82125,5.3522,5.86985,1,2012-06-22 01:30:00,1,4,6,174
3,5.0909,0.33387,32.364,3.5567,-13.9460,0.63822,0.12357,0.008703,-0.035944,-0.124000,...,0.986102,4.82125,5.3522,5.86985,1,2012-06-22 02:00:00,2,4,6,174
4,5.8500,0.44881,28.569,3.8587,-14.0660,0.77249,0.15957,0.011172,0.074542,0.005818,...,0.986102,4.82125,5.3522,5.86985,1,2012-06-22 02:30:00,2,4,6,174
5,4.1455,0.98754,24.648,12.6020,-14.1820,0.76101,0.13249,0.009131,0.025845,0.143340,...,0.986102,4.82125,5.3522,5.86985,1,2012-06-22 03:00:00,3,4,6,174
6,3.4408,0.80687,11.048,8.1544,-14.9320,0.80979,0.13693,0.009080,0.201620,0.302600,...,0.986102,4.82125,5.3522,5.86985,1,2012-06-22 03:30:00,3,4,6,174
7,3.4772,0.80788,36.742,8.4677,-15.0720,1.20000,0.13514,0.008611,-0.022391,0.281180,...,0.986102,4.82125,5.3522,5.86985,2,2012-06-22 04:00:00,4,4,6,174
8,5.4707,0.66564,13.321,5.3211,-12.6100,0.75674,0.14830,0.009997,0.147020,0.222110,...,0.986102,4.82125,5.3522,5.86985,2,2012-06-22 04:30:00,4,4,6,174
9,4.8830,0.53958,14.134,3.1002,-11.6310,1.00730,0.11883,0.007229,-0.068287,0.151030,...,0.986102,4.82125,5.3522,5.86985,2,2012-06-22 05:00:00,5,4,6,174


### Sequence Data

In [9]:
# Features for training
drivers = ['LE', 'H', 'Tau', 'LE_strg', 'Ta', 'RH', 'VPD', 'Rg', 'Ustar', 'Tsoil1', 'Tsoil2']

#-------------------------------------------------
# prepare and split data for regressor
columns_to_pick = drivers + stat_tags + rg_tag + ['hour', 'dayofweek', 'month', 'dayofyear']

In [ ]:
def prepare_lstm_data_version2(data, sequence_type, gap_type):
    """
    Prepare data for LSTM where the sequence data includes the gap.
    
    Parameters:
    data (pd.DataFrame): DataFrame with extracted features and NEE values.
    sequence_type (str): Type of sequence length ('very short', 'short', 'medium', 'week', 'long', 'month').
    gap_type (str): Type of gap to insert ('very short', 'short', 'medium', 'week', 'long', 'month').
    
    Returns:
    np.array: Prepared input sequences for LSTM.
    np.array: Corresponding output sequences for LSTM.
    """
    time_deltas = {
        "very short": pd.Timedelta(minutes=30),
        "short": pd.Timedelta(hours=4),
        "medium": pd.Timedelta(hours=32),
        "week": pd.Timedelta(days=7),
        "long": pd.Timedelta(days=12),
        "month": pd.Timedelta(days=30)
    }
    
    sequence_length = int(time_deltas[sequence_type].total_seconds() // 1800)  # Convert to number of half-hours
    gap_length = time_deltas[gap_type]
    
    sequences = []
    targets = []
    
    for i in range(len(data) - sequence_length):
        seq_end_time = data['DateTime'].iloc[i + sequence_length - 1]
        target_end_time = seq_end_time + gap_length
        
        if target_end_time > data['DateTime'].iloc[-1]:
            break
        
        seq = data.iloc[i:i + sequence_length]
        target = data.loc[(data['DateTime'] > seq_end_time) & (data['DateTime'] <= target_end_time), 'NEE'].values
        
        if len(target) > 0 and not np.isnan(target).any():
            sequences.append(seq)
            targets.append(target)
    
    X = np.array([seq.drop(columns=['DateTime']).values for seq in sequences])
    y = targets  # Keeping the targets as a list to handle dynamic lengths
    
    return X, y

# Example Usage for Version 2
X_train, Y_train = prepare_lstm_data_version2(data, sequence_type="week", gap_type="week")


In [18]:
X

array([], dtype=float64)